In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

# jax
import jax.numpy as jnp
import time

import sys
import os
import importlib
import pickle
import itertools

path_ = os.path.abspath(os.path.join('..', '00_utils'))
if path_ not in sys.path:
    sys.path.append(path_)

path_ = os.path.abspath(os.path.join('..', '00_models'))
if path_ not in sys.path:
    sys.path.append(path_)

path_ = os.path.abspath(os.path.join('..', '00_utils_training'))
if path_ not in sys.path:
    sys.path.append(path_)

import logging
logging.basicConfig(level=logging.ERROR, filename='error_log.txt')

import optimize_pyomo_rl

* [X] Regularization Strength with Fixed Tolerance
* [X] 'tol' with no regularization
* [X] 'tol - 'reg' cross-validation

In [2]:
param_combinations_input_features = [{'prev_hour':True, 'prev_week':True, 'prev_year':True, 'm':1,'ls': [8, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':True, 'prev_year':False, 'm':1, 'ls': [7, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':True, 'm':1, 'ls': [7, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':False, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':True, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':True, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':False, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':False, 'm':1, 'ls': [5, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':True, 'prev_year':True, 'm':0,'ls': [7, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':True, 'prev_year':False, 'm':0, 'ls': [6, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':True, 'm':0, 'ls': [6, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':False, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':True, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':True, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':False, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':False, 'm':0, 'ls': [4, 32, 1]}, 
                        ]

In [19]:
importlib.reload(optimize_pyomo_rl)
ExperimentRunner = optimize_pyomo_rl.ExperimentRunner

In [3]:
help(ExperimentRunner)

Help on class ExperimentRunner in module optimize_pyomo_rl:

class ExperimentRunner(builtins.object)
 |  ExperimentRunner(start_date, optimization_aim, extra_inputs={})
 |  
 |  extra_inputs: dict. Expected keys:
 |      layer_sizes; default: [6, 32, 1]
 |      penalty; default: 1e-5
 |      params_data; default: generated by default_data_params(start_date)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, start_date, optimization_aim, extra_inputs={})
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  collect_metrics(self, experiment_results)
 |  
 |  define_param_combinations(self)
 |      Define the parameter combinations to be used in the optimization;
 |      based on the optimization aim.
 |  
 |  initialize_inputs(self)
 |  
 |  initialize_metrics(self)
 |  
 |  run(self)
 |  
 |  save_results(self, description)
 |  
 |  update_date(self, date, param_comb, file, iter)
 |  
 |  update_prams(self, param_comb)
 |  
 |  -----------------------------

In [4]:
# extra_inputs 

In [20]:
start_date = '2015-01-01'
extra_input = {}
extra_input['params_data'] = {'file_path': '../00_data/df_train.csv', 'start_date': start_date, 
                'n_points': 400, 'split': 200, 'n_days': 1, 'm': 0, 
                'prev_hour': True, 'prev_week': True, 'prev_year': False,
                'spacing': 'gauss_radau',
                'encoding': {'settlement_date': 't', 'temperature': 'var1', 'hour': 'var2', 'nd': 'y'},}

extra_input['params_sequence'] = {'sequence_len': 1, 'frequency': 1}
extra_input['params_model'] = {'layer_sizes': [6, 32, 1], 'penalty': 1e-5}
extra_input['params_solver'] = { "tol" : 1e-4, 
                         #"dual_inf_tol": 1e-4, #"compl_inf_tol": 1e-3, #"constr_viol_tol": 1e-5, 
                         #"bound_relax_factor": 0.0001, 
                         "acceptable_constr_viol_tol": 1e-15, "acceptable_dual_inf_tol": 1e-15, "acceptable_compl_inf_tol": 1e-15, 
                         "halt_on_ampl_error" : 'yes', "print_level": 1, "max_iter": 500, 'warm_start_init_point': 'yes'}

runner = ExperimentRunner(start_date, 'tolerance_mix', extra_input)
runner.run()

Generating default parameters for ode solver
Running iteration 1 with parameters: 5
current_16_08
y
Ipopt 3.14.16: tol=0.0001
acceptable_constr_viol_tol=1e-15
acceptable_dual_inf_tol=1e-15
acceptable_compl_inf_tol=1e-15
halt_on_ampl_error=yes
print_level=1
max_iter=5
warm_start_init_point=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Ipopt 3.14.16: print_level=1


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github

In [21]:
runner.results_full

{(5,
  '2018-01-01'): {'result': {'solver_time': 0.4677138328552246,
   'termination_condition': <TerminationCondition.maxIterations: 'maxIterations'>,
   'message': 'Ipopt 3.14.16\\x3a Maximum Number of Iterations Exceeded.'}, 'times_elapsed': 0.4677138328552246, 'mse_odeint': 0.6079987396384379, 'mse_coll_ode': 1.4354062045745395, 'mse_odeint_test': 4.430379824307327, 'mse_coll_ode_test': 2.9862324340527397},
 (10,
  '2018-01-01'): {'result': {'solver_time': 0.582740068435669,
   'termination_condition': <TerminationCondition.maxIterations: 'maxIterations'>,
   'message': 'Ipopt 3.14.16\\x3a Maximum Number of Iterations Exceeded.'}, 'times_elapsed': 0.582740068435669, 'mse_odeint': 0.5568337721463373, 'mse_coll_ode': 1.6136182941993895, 'mse_odeint_test': 39.12623471023522, 'mse_coll_ode_test': 10.19917131604091},
 (15,
  '2018-01-01'): {'result': {'solver_time': 0.8814189434051514,
   'termination_condition': <TerminationCondition.maxIterations: 'maxIterations'>,
   'message': 'Ipop

In [39]:
runner.save_results('tol_reg_radau')

Results saved to results/tol_reg_radau_2024-08-23_12-10-48_full.pkl
Results saved to results/tol_reg_radau_2024-08-23_12-10-48_avg.pkl
